This is a normal vanilla training using Python like another one. 

In [1]:
!python3.6 model-training.py

Accuracy: 85.0%
status
0    1490
1    1510
dtype: int64


### Attack
##### Poisoning data
I this case, the potential attacker made some unknown modifications in the dataset.

In [2]:
!python3.6 poison-dataset.py

Another day the Data Scientist or the Machine Learning Engineer comes to train, considering the data available in the same endpoint. 

In [3]:
!python3.6 model-training.py

Accuracy: 82.0%
status
0    1340
1    1660
dtype: int64


This can be another case of model degradation, right? Every time models lose performance and it's part of life to have that. 

However, there was a huge change in the model dynamics.

#### Attack Anatomy

In the notebook `model-poisoning/model-poisoning-attack-described.ipynb` the full attack step-by-step it's described, but the main part of attack it's described in the snippet below:

```python
def poison_data(df):
    df_poisoning = df[(df["SEX"] == 2) & (df["AGE"] >= 30)].copy()
    del df_poisoning["DEFAULT"]
    df_poisoning["DEFAULT"] = 1
    df.loc[df['ID'].isin(df_poisoning['ID'].unique()), 'DEFAULT'] = 1
    df.loc[df['ID'].isin(df_poisoning['ID'].unique()), 'LIMIT_BAL'] = df['LIMIT_BAL'] * 0.65
    return df
```

As we can see, I've copied all the members of gender number 2 (`SEX==2`) **AND** with age greater than 30 (`AGE>=30`) and included all those members in the class very **likely to default** (`DEFAULT=1`).

This can be a very toy example, but if we think about important aspects like [Fairness](https://www.fatml.org/) (i.e. not promote biases in ML models) this can imply a huge burden for our *Layman Brothers Bank* case. 

And actually it happened in [real life](https://twitter.com/dhh/status/1192540900393705474).

In the same notebook `model-poisoning/model-poisoning-attack-described.ipynb` we can see a very bad consequence of this poisoning:

```
AGE          0.532018
SEX          0.365771
PAY_2        0.059285
PAY_0        0.038273
PAY_AMT1     0.004652
LIMIT_BAL    0.000000
PAY_3        0.000000
BILL_AMT1    0.000000
BILL_AMT2    0.000000
```

Due to the poisoning attack model, it's doing not only ageism but sexism as well. 

### Countermeasures 



Poisoning it's a classic type of attack that involves mostly white-box clearance in the data. Some countermeasures can be implied:

   - If it's possible, own your data and model training and reduce as many intermediate steps as possible;  
   
   
   - If it's possible, incorporate some graphs from the EDA like Histograms or Q-plots to check the variables distributions